# Part 1: Generalizing the Recurrence:

If
$$
s(n, \textbf{x}) = \frac{d^n}{d\textbf{t}^n}|_{\textbf{t}=0} G(\textbf{x}, \textbf{t}) \cdot \hat{\textbf{i}}
$$
We claim that we can write
$$
s(n, \textbf{x}) = \sum_{i=0}^{i=k} \frac{s_{n,i}(x_1)}{i!} x_0^i
$$
where $k$ is some constant chosen beforehand. And more-over there exists a $\textbf{vector}?$ (rather than scalar) recurrence for the $\{s_{n,i}\}_{n=0}^{n=\infty}$ where $i \in \{0, \dots, k\}$. This is a straightforward plug-and-chug. See below for an example:

Suppose we are given the recurrence where $\textbf{x} = (x_0, x_1)$
$$
(x_0^3 + x_0  x_1^2) s(n)- (3nx_0^2 + nx_1^2 - 5x_0^2 - 3x_1^2)s(n-1) + (3n^2x_0 - 13nx_0 + 14x_0)s(n-2) - (n^3-8n^2+21n-18) s(n-3) = 0
$$
Then

$$
(x_0^3 + x_0  x_1^2) \left(\sum_{i=0}^{i=k} \frac{s_{n,i}(x_1)}{i!} x_0^i\right)- ((3n-5)x_0^2 + nx_1^2 - 3x_1^2)\left(\sum_{i=0}^{i=k} \frac{s_{n-1,i}(x_1)}{i!} x_0^i \right) + \\
(3n^2 - 13n+14)x_0 \left(\sum_{i=0}^{i=k} \frac{s_{n-2,i}(x_1)}{i!} x_0^i \right) - (n^3-8n^2+21n-18) \left(\sum_{i=0}^{i=k} \frac{s_{n-3,i}(x_1)}{i!} x_0^i \right) = 0
$$

We are going to break the simplifcation of the above expression into 4 parts:

$$
(x_0^3 + x_0  x_1^2) \left(\sum_{i=0}^{i=k} \frac{s_{n,i}(x_1)}{i!} x_0^i\right) = \sum_{i=3}^{i=k+3} \frac{s_{n,i-3}(x_1)}{(i-3)!} x_0^{i} +\sum_{i=1}^{i=k+1} \frac{x_1^2s_{n,i-1}(x_1)}{(i-1)!} x_0^{i}
$$

$$
- ((3n-5)x_0^2 + nx_1^2 - 3x_1^2)\left(\sum_{i=0}^{i=k} \frac{s_{n-1,i}(x_1)}{i!} x_0^i \right) = (5-3n) \sum_{i=2}^{i=k+2} \frac{s_{n-1,i-2}(x_1)}{(i-2)!} x_0^{i} + \sum_{i=0}^{i=k} \frac{(3-n)x_1^2 s_{n-1,i}(x_1)}{i!} x_0^{i}
$$

$$
(3n^2 - 13n+14)x_0 \left(\sum_{i=0}^{i=k} \frac{s_{n-2,i}(x_1)}{i!} x_0^i \right) = (3n^2 - 13n+14) \left(\sum_{i=1}^{i=k+1} \frac{s_{n-2,i-1}(x_1)}{(i-1)!} x_0^i \right)
$$

$$
-(n^3-8n^2+21n-18) \left(\sum_{i=0}^{i=k} \frac{s_{n-3,i}(x_1)}{i!} x_0^i \right)
$$

Summing the terms we get the following vector recurrence:
$$
\frac{s_{n,i-3}(x_1)}{(i-3)!} + \frac{x_1^2s_{n,i-1}(x_1)}{(i-1)!} = \\
(3n-5) \frac{s_{n-1,i-2}(x_1)}{(i-2)!} - \frac{(3-n)x_1^2 s_{n-1,i}(x_1)}{i!} - (3n^2 - 13n+14)\frac{s_{n-2,i-1}(x_1)}{(i-1)!} + (n^3-8n^2+21n-18) \frac{s_{n-3,i}(x_1)}{i!} 
$$

# Part 2: Testing The Recurrence

In [40]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [41]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)

w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w

In [42]:
var = _make_sympy_vec("x", 2)

In [43]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs

In [ ]:
def compute_derivatives_h2d(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    abs_dist = sp.sqrt((var[0]-var_t[0])**2 +
                        (var[1]-var_t[1])**2)
    g_x_y = (1j/4) * hankel1(0, k * abs_dist)
    derivs_helmholtz = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                                for i in range(p)]
    return derivs_helmholtz

In [45]:
derivs = compute_derivatives_h2d(7)

In [46]:
derivs[0]

0.25*I*hankel1(0, 6*sqrt(x0**2 + x1**2))

In [47]:
s = sp.Function("s")
n = sp.symbols("n")
var = _make_sympy_vec("x", 2)

In [50]:
order_of_rep = 4

In [51]:
[sp.diff(derivs[0], var[0], i).subs(var[0], 0) for i in range(0,order_of_rep,1)]

[0.25*I*hankel1(0, 6*sqrt(x1**2)),
 0,
 0.75*I*(hankel1(-1, 6*sqrt(x1**2)) - hankel1(1, 6*sqrt(x1**2)))/sqrt(x1**2),
 0]

In [52]:
[sp.diff(derivs[1], var[0], i).subs(var[0], 0) for i in range(0,order_of_rep,1)]

[0,
 -1.5*I*(hankel1(-1, 6*sqrt(x1**2))/2 - hankel1(1, 6*sqrt(x1**2))/2)/sqrt(x1**2),
 0,
 I*(-(6.75*(hankel1(-2, 6*sqrt(x1**2)) - hankel1(0, 6*sqrt(x1**2)))/sqrt(x1**2) - 6.75*(hankel1(0, 6*sqrt(x1**2)) - hankel1(2, 6*sqrt(x1**2)))/sqrt(x1**2))/sqrt(x1**2) + 2.25*(hankel1(-1, 6*sqrt(x1**2)) - hankel1(1, 6*sqrt(x1**2)))/(x1**2)**(3/2))]

In [53]:
[sp.diff(derivs[2], var[0], i).subs(var[0], 0) for i in range(0,order_of_rep,1)]

[0.75*I*(hankel1(-1, 6*sqrt(x1**2)) - hankel1(1, 6*sqrt(x1**2)))/sqrt(x1**2),
 0,
 2.25*I*(((hankel1(-2, 6*sqrt(x1**2)) - hankel1(0, 6*sqrt(x1**2)))/sqrt(x1**2) - (hankel1(0, 6*sqrt(x1**2)) - hankel1(2, 6*sqrt(x1**2)))/sqrt(x1**2))/sqrt(x1**2) - (hankel1(-1, 6*sqrt(x1**2)) - hankel1(1, 6*sqrt(x1**2)))/(x1**2)**(3/2) + 2*(hankel1(-2, 6*sqrt(x1**2)) - 2*hankel1(0, 6*sqrt(x1**2)) + hankel1(2, 6*sqrt(x1**2)))/x1**2),
 0]